# Implementing our RAG Application

We've walked through the process of ingesting our data. Now we want to setup our search application.

There are multiple ways of implementing search that we won't be addressing in this workshop.

We're going to implement a similarity search. We can choose to use the OpenSearch®️ SDK or we can use LangChain.

## Similarity Search with Langchain

LangChain allows us to connect our existing configurations across our platform to ensure that we settings are consistent. This is paramount to the success of your similarity search in that if you're search parameters differ from the values you selected to load data from, you can run into some issues.

LangChain gives us the ability to access a [preexisting OpenSearch instance](https://python.langchain.com/docs/integrations/vectorstores/opensearch/#using-a-preexisting-opensearch-instance).

In [53]:
import os
from pprint import pprint

from dotenv import load_dotenv
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import OpenSearchVectorSearch

load_dotenv()

embeddings = HuggingFaceEmbeddings()
vector_search = OpenSearchVectorSearch(
    index_name=os.getenv("INDEX_NAME"),
    embedding_function=embeddings,
    opensearch_url=os.getenv("OPENSEARCH_SERVICE_URI"),
)

query = "How can I be productive"

results = vector_search.similarity_search(
    query,
    vector_field="content_vector",
    text_field="content",
    metadata_field="*",
)

pprint(results[0].page_content)

/Users/jay.miller/preparing-data-for-opensearch-and-rag/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('. Boy, how do you do? I feel you on this ever since I started just having a '
 'single page to write down what actually gets done and crossing it off, even '
 "if it doesn't get on the list until after I've done something. It's helped "
 "me realize and recognize how much I'm actually getting done")


In [21]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOllama(model="llama3")
prompt = ChatPromptTemplate.from_template("""
        Offer supportive advice for the question {query} with supporting quotes from 
        "{docs}".
                                          
        Don't include quotes from other sources.
""")

chain = prompt | llm | StrOutputParser()

print(chain.invoke({"query": query, "docs": "\n".join([result.page_content for result in results])}))
print("-------------------")
print(f'Here are some episodes that might help you with "{query}":')
episodes = set()
for result in results:
    episodes.add(f"{result.metadata["title"]} - {result.metadata["url"]}")
print("\n".join(episodes))

The art of setting boundaries and saying no! It's a crucial skill to master, especially in today's fast-paced world where everyone wants a piece of your time and attention.

Firstly, remember that saying no is not about being selfish or rude. It's about taking care of yourself and prioritizing what's truly important to you. As I've learned, most people are happy to respect your boundaries once they're explicitly communicated. So, don't be afraid to verbalize what you need or don't want from others.

To set effective boundaries, identify the limits you want to establish and communicate them clearly. For example, if you have a meeting on Wednesday, say no to social invitations that conflict with it. And remember, exceptions are okay! Life is unpredictable, and sometimes things come up that require adjustments to your plans.

Here's another tip: create a buffer period before committing to anything new. Give yourself time to think about whether the request aligns with your priorities and v

This is good but we probably don't want to use Ollama in our production environment. Let look at how easy it is to use a different model.

> NOTE: ⚠️ The next block uses the OpenAI API which cannot be used without an API key which you will need to pay for.

In [55]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

load_dotenv()

llm = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages([
    ("system",
     """Offer supportive advice for the question {query} with supporting quotes from 
     ---
     {docs}
     ---
     Wrap quotes in quotation marks. Don't include quotes from other sources.
     If there are no documents to quote, say \"I don't have any information on that.\"""")
    ,
    ("user",
     "{query}"),
])

chain = prompt | llm | StrOutputParser()

print(chain.invoke({"query": query, "docs": "\n".join([result.page_content for result in results])}))
print("-------------------")
print(f'Here are some episodes that might help you with "{query}":')
episodes = set()
for result in results:
    episodes.add(f"{result.metadata["title"]} - {result.metadata["url"]}")
print("\n".join(episodes))

To be productive, it's important to prioritize and focus on what truly matters. One helpful strategy is to create a daily to-do list and cross off tasks as you complete them. This can help you stay organized and motivated. As one person mentioned, "ever since I started just having a single page to write down what actually gets done and crossing it off, even if it doesn't get on the list until after I've done something. It's helped me realize and recognize how much I'm actually getting done."

Setting boundaries with others is also crucial for productivity. Learn to say no to interruptions that are not urgent or important during your peak focus times. As another person shared, "I need to set boundaries for others as I'm constantly interrupted in the depths of my work or high focus times with things that are not urgent."

When it comes to dealing with external factors like other people, it's essential to find a balance between being helpful and protecting your own time and focus. As ment